In [1]:
%load_ext autoreload
%autoreload 2

# import libraries
import pandas as pd
import re

import os

# NLP
import nltk
nltk.download('punkt')                            # necesario para que funcione el tokenizador
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


## Cargar datos
Se cargan los datos de todas las hojas del espacio de trabajo del documento excel. La variable resultante es un diccionario que tiene como "keys" cada hoja del excel (nombre del profesor) y como "values" los contenidos de cada hoja.

In [3]:
df = pd.read_excel("data/Proyecto. Dep- Física.xlsx")
df = df.dropna(subset=["Descripción original"])

In [4]:
df.head(3)

,Título original,Título en español,Título en ingles,Descripción original,Descripción en español,Descripción en ingles,Citaciones,Idioma,Fecha,Tipo,Fuente,Enlace
0,XPS structure analysis of TiN/TiC bilayers pro...,XPS structure analysis of TiN/TiC bilayers pro...,XPS structure analysis of TiN/TiC bilayers pro...,TiN/TiC Bilayers were grown on 304 stainless s...,Los bicapas de estaño / tic se cultivaron en 3...,TiN/TiC Bilayers were grown on 304 stainless s...,28,en,2010-01-01,Artículo de revista,SCOPUS,https://www.scopus.com/inward/record.uri?partn...
3,XPS structure analysis of TiN/TiC bilayers pro...,Análisis de la estructura XPS de las bicapas d...,XPS structure analysis of TiN/TiC bilayers pro...,Tin / ICT Bilacas were grew on stainless steel...,se crecieron bicapas de TiN/TiC sobre sustrato...,Tin / ICT Bilacas were grew on stainless steel...,23,en,2010-09-01,Artículo de revista,RG,https://www.researchgate.net/publication/49598...
5,Comparative study between natural and syntheti...,Estudio comparativo entre hidroxiapatita natur...,Comparative study between natural and syntheti...,"In this work, a study of hydroxyapatite (HAp) ...","En este trabajo, se realizó un estudio de pol...","In this work, a study of hydroxyapatite (HAp) ...",17,en,2018-01-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.1590/s1517-707620180004.0551


In [5]:
df.iloc[0,3]

'TiN/TiC Bilayers were grown on 304 stainless steel substrates using physical vapour deposition assisted by pulsed arc plasma system (PAPVD) at two substrate temperatures (50°C and 150°C). X ray photoelectron spectroscopy (XPS) was used to analyze the chemical composition by observing the behaviour of the Ti2p, N1s and C1s lines. Binding energy analysis confirmed TiN and TiC formation. The C1s and Ti2p peaks shifted with increasing XPS sputtering time, thus revealing hydrocarbides contamination. Furthermore, depth profiles of the TiN/TiC bilayers showed that the films grown at a substrate temperature of 150°C had a thicker TiN layer than the samples grown at 50°C. Nitrogen had diffused into the TiC layer and carbon into the TiN layer in both films.'

In [8]:
# cargar datos de todas las hojas
data = pd.read_excel("data/Proyecto. Dep- Física.xlsx", sheet_name=None)

# Separar información util
names = [name for name in data.keys()]
# titulos

In [9]:
data['Arango Arango Pedro Jose'].columns

Index(['Título original', 'Título en español', 'Título en ingles',
       'Descripción original', 'Descripción en español',
       'Descripción en ingles', 'Citaciones', 'Idioma', 'Fecha', 'Tipo',
       'Fuente', 'Enlace'],
      dtype='object')

In [11]:
tokens = []
for name in names:
    # print(name)
    df = data[name]
    df.dropna(subset=["Descripción original"], inplace =True)                # quitar filas de datos faltantes                    
    # df.iloc[:,3] = df.iloc[:,3].map(lambda x: re.sub('[,\.!:?]', '', x))     # quitar puntuación (TENER CUIDADO!!!)
    # df.iloc[:,3] = df.iloc[:,3].map(lambda x: x.lower())                     # llevar todo a minusculas, luego de tokenizar.
    # tokens.append(df.iloc[:,3].apply(lambda x: word_tokenize(x)).to_list())
    
    

In [12]:
df

,Título original,Título en español,Título en ingles,Descripción original,Descripción en español,Descripción en ingles,Citaciones,Idioma,Fecha,Tipo,Fuente,Enlace
0,Coagulation-flocculation sequential with Fento...,COAGULA-CLOCTULATE SECUENCIA CON PROCESOS DE F...,Coagulation-flocculation sequential with Fento...,"During the C-F, Al2(SO4)3 was used as coagula...","Durante el C-F, AL2 (SO4) 3 se usó como coagu...","During the C-F, Al2(SO4)3 was used as coagula...",96.0,en,2017-04-15,Artículo de revista,SCOPUS,http://dx.doi.org/10.1016/j.jenvman.2017.01.015
2,Optimization and toxicity assessment of a comb...,Evaluación de la optimización y toxicidad de u...,Optimization and toxicity assessment of a comb...,"V.In this study, the potential application of ...",Aguas materiales textiles resultantes de una ...,"V.In this study, the potential application of ...",50.0,en,2019-02-15,Artículo de revista,SCOPUS,http://dx.doi.org/10.1016/j.scitotenv.2018.09.125
4,Carbon monoxide oxidation over Au/Ce1-xZrxO 2 ...,Oxidación de monóxido de carbono sobre cataliz...,Carbon monoxide oxidation over Au/Ce1-xZrxO 2 ...,"The Au/Ce1-xZrxO2 (x = 0, 0.25, 1) catalysts w...","Los catalizadores AU / CE1-XZRXO2 (X = 0, 0.25...","The Au/Ce1-xZrxO2 (x = 0, 0.25, 1) catalysts w...",33.0,en,2009-01-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.1007/s10562-008-9749-1
5,Kinetic study on the catalytic esterification ...,Estudio cinético sobre la esterificación catal...,Kinetic study on the catalytic esterification ...,A kinetic model was fitted for the liquid phas...,Se instaló un modelo cinético para la esterifi...,A kinetic model was fitted for the liquid phas...,26.0,en,2013-01-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.1016/j.ces.2013.07.009
6,Optimization of sequential chemical coagulatio...,Optimización de la coagulación química secuenc...,Optimization of sequential chemical coagulatio...,Complete characterization of the effluent was...,La caracterización completa del efluente se r...,Complete characterization of the effluent was...,26.0,en,2018-04-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.1016/j.jwpe.2018.01.005
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Parametric sensitivity analysis of o-nitroanil...,Parametric sensitivity analysis of o-nitroanil...,Parametric sensitivity analysis of o-nitroanil...,"All Rights Reserved.In this work, the paramet...","En este trabajo, se estudió la sensibilidad p...","All Rights Reserved.In this work, the paramet...",0.0,en,2019-02-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.4067/S0718-07642019000100015
78,Enhanced adsorption and desorption of Cr(VI) f...,Adsorción mejorada y desorción de CR (VI) de l...,Enhanced adsorption and desorption of Cr(VI) f...,Hydrous Ce1–x Zr x O2 (x = 0.25; 0.5; 0.75) mi...,HYDROUS CE1-X ZR X O2 (x = 0.25; 0.5; 0.75) Se...,Hydrous Ce1–x Zr x O2 (x = 0.25; 0.5; 0.75) mi...,0.0,en,2020-01-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.1080/01932691.2020.1845716
79,An adaptive WENO algorithm for one-dimensional...,Un algoritmo de Weno Adaptable para ecuaciones...,An adaptive WENO algorithm for one-dimensional...,The proposed algorithm uses a fully adaptive ...,El algoritmo propuesto utiliza un esquema tot...,The proposed algorithm uses a fully adaptive ...,0.0,en,2020-02-23,Artículo de revista,SCOPUS,http://dx.doi.org/10.1016/j.ces.2019.115391
80,Parametric sensitivity analysis for the indust...,Análisis de sensibilidad paramétrica para el c...,Parametric sensitivity analysis for the indust...,"Licensee MDPI, Basel, Switzerland.The operati...","Licenciatario MDPI, Basilea, Suiza. La operac...","Licensee MDPI, Basel, Switzerland.The operati...",0.0,en,2020-06-01,Artículo de revista,SCOPUS,http://dx.doi.org/10.3390/catal10060626


In [55]:
test = df.iloc[0,3]
test

'TiN/TiC Bilayers were grown on 304 stainless steel substrates using physical vapour deposition assisted by pulsed arc plasma system (PAPVD) at two substrate temperatures (50°C and 150°C). X ray photoelectron spectroscopy (XPS) was used to analyze the chemical composition by observing the behaviour of the Ti2p, N1s and C1s lines. Binding energy analysis confirmed TiN and TiC formation. The C1s and Ti2p peaks shifted with increasing XPS sputtering time, thus revealing hydrocarbides contamination. Furthermore, depth profiles of the TiN/TiC bilayers showed that the films grown at a substrate temperature of 150°C had a thicker TiN layer than the samples grown at 50°C. Nitrogen had diffused into the TiC layer and carbon into the TiN layer in both films.'

In [48]:
tok = df.iloc[:,3].apply(lambda x: word_tokenize(x)).to_list()

In [49]:
type(tok)

list

## Preprocesamiento

Se llevaran a cabo los siguientes pasos:

    - Tokenización:
    - Palabras con menos de tres caracteres se retiran.
    - Remover Stopwords.
    - Lematización:
    - Stemmed:


In [9]:
pd.__file__

'C:\\Users\\Usuario\\anaconda3\\lib\\site-packages\\pandas\\__init__.py'